# 생성한 데이터셋 업로드

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

In [ ]:
import pandas as pd

df = pd.read_csv("data/ragas_synthetic_dataset.csv")
df.head()

## DeepL 을 활용한 번역

- [DeepL API 키 발급](https://www.deepl.com/ko/your-account/keys)

발급 받은 키를 `.env` 파일에 `DEEPL_API_KEY` 로 저장합니다.

In [ ]:
# !pip install -qU deepl

In [ ]:
import os
from langchain_teddynote.translate import Translator

# api키 설정
deepl_api_key = os.getenv("DEEPL_API_KEY")

# 객체 생성
translator = Translator(deepl_api_key, "EN", "KO")

# 번역 실행
translated_text = translator("hello, nice to meet you")
print(translated_text)

In [ ]:
from tqdm import tqdm

# 번역
for i, row in tqdm(df.iterrows(), total=len(df), desc="번역 진행 중"):
    df.loc[i, "question_translated"] = translator(row["question"])
    df.loc[i, "ground_truth_translated"] = translator(row["ground_truth"])

In [ ]:
df.head()

번역 전

In [ ]:
# question, ground_truth 열을 확인합니다.
df.loc[:, ["question", "ground_truth"]].head()

번역한 결과를 확인합니다.

In [ ]:
# question_translated, ground_truth_translated 열을 확인합니다.
df.loc[:, ["question_translated", "ground_truth_translated"]].head()

In [ ]:
# question, ground_truth 열을 삭제하고 question_translated, ground_truth_translated 열의 이름을 변경합니다.
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()

In [ ]:
# 번역한 데이터셋을 저장합니다.
df.to_csv("data/ragas_synthetic_dataset_translated.csv", index=False)

## Hugging Face Dataset 업로드

Pandas DataFrame 을 Hugging Face Dataset 으로 변환하여 업로드를 진행합니다.

In [ ]:
from datasets import Dataset

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 확인
print(dataset)

In [ ]:
from datasets import Dataset
import os

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 이름 설정 (원하는 이름으로 변경하세요)
dataset_name = "teddylee777/ragas-test-dataset"

# 데이터셋 업로드
dataset.push_to_hub(
    dataset_name,
    private=True,  # private=False로 설정하면 공개 데이터셋이 됩니다.
    split="korean_v1",  # 데이터셋 split 이름 입력
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

[참고] Dataset Viewer 는 표기되기까지 시간이 좀 걸릴 수 있습니다.